In [ ]:
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from pydantic import ValidationError
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

In [ ]:


class ArticleDraft(BaseModel):
    title: str = Field(description="Title of the article")
    introduction: str = Field(description="Opening paragraph")
    body: str = Field(description="Main content")
    quotes: List = Field(description="Verified quotes or facts (must be self-contained strings without unescaped quotes)")
    conclusion: str = Field(description="Closing paragraph")

In [ ]:

def writer_node(state):

    parser = PydanticOutputParser(pydantic_object=ArticleDraft)

    format_instructions = parser.get_format_instructions()

    max_retries = 3
    attempt = 0

    agent = create_react_agent(
        model=llm,
        tools=[ddg_search_tool,retrieve_context],
        prompt=(
            "You are a **Research Assistant**.\n"
            "For each query, you MUST use both tools in a logical sequence:\n"
            "1) Use **retrieve_context** to fetch internal context.\n"
            "2) Use **DuckDuckGoSearch** to gather up-to-date web facts.\n\n"
            "Your output MUST conform to this JSON schema exactly (no extra fields, keys must match):\n"
            f"{format_instructions}\n\n"
            "During your reasoning, label steps as Thought, Action, Observation, "
            "and end with Final Answer formatted exactly as the above JSON schema.\n\n"
        )
    )

    user_msg = {"role": "user", "content": f"Research this topic in detail: {state['topic']}"}

    while attempt < max_retries:
        
        attempt += 1
        
        
        # Run agent and capture full assistant output (stream or no-stream)
        llm_response = agent.invoke({"messages": [user_msg]})
        # assistant_msg = llm_response["messages"][-1]
        # ai_content = assistant_msg.content

        ai_content = ""
        for step in agent.stream({"messages": [user_msg]}, stream_mode="values"):
            msg = step["messages"][-1]
            msg.pretty_print()
            # Capture only if it's an assistant message
            if isinstance(msg, AIMessage):
                ai_content = msg.content
        
        
        try :
            # Parse the final JSON into Pydantic model
            article: ArticleDraft = parser.parse(ai_content)
            return article
        except ValidationError as e:
            print(f"[Attempt {attempt}] Parsing failed:", e)
            # Optionally modify the prompt to highlight the error:
            prompt += "\n\nNote: Your previous output did not match the required JSON schema. Please fix it exactly."
            continue

    # If all attempts fail, raise or return empty/default
    raise RuntimeError(f"Failed to get valid ArticleDraft JSON after {max_retries} attempts.")


In [ ]:
res = writer_node({"topic": "Impact of Dumbledore in Harry's life ?"})

================================ Human Message =================================

Research this topic in detail: Impact of Dumbledore in Harry's life ?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (bmxn10v8f)
 Call ID: bmxn10v8f
  Args:
    n_docs: 5
    query: Impact of Dumbledore on Harry's life
  DuckDuckGoSearch (y0m7wxd78)
 Call ID: y0m7wxd78
  Args:
    __arg1: Impact of Dumbledore on Harry's life
================================= Tool Message =================================
Name: DuckDuckGoSearch

snippet: When we think of Harry Potter, it's impossible not to think of Albus Dumbledore. As the wise and enigmatic headmaster of Hogwarts, Dumbledore played a pivotal role in shaping not just Harry's destiny, but the entire wizarding world. Today, I want to dive deep into Dumbledore's character and explore how his actions and decisions molded Harry into the hero we all know and love. Buckle up ..., title: Dumbledore'

In [ ]:
res

ArticleDraft(title="The Enduring Impact of Albus Dumbledore on Harry Potter's Life", introduction="Albus Dumbledore, the enigmatic headmaster of Hogwarts, served as both mentor and moral compass in Harry Potter's life. Through strategic guidance and profound wisdom, Dumbledore shaped Harry's understanding of love, sacrifice, and the complexities of good versus evil throughout the series.", body="Dumbledore's influence manifested in three key ways: 1) He provided Harry with critical knowledge about Voldemort's past and the Horcruxes, 2) He nurtured Harry's emotional growth by sharing personal vulnerabilities about his brother Aberforth, and 3) He established the Order of the Phoenix to protect Harry while preparing him for his ultimate destiny. The final conversation between Dumbledore and Harry in 'The Deathly Hallows' emphasized philosophical truths about perception and reality ('Of course it is happening inside your head, Harry, but why on earth should that mean it is not real?'). Th

In [ ]:
from typing import List, Literal

In [ ]:
class Classify(BaseModel):
    classification: Literal["generic", "fda"] = Field(
        description="Classify the topic type: 'generic' or 'fda'"
    )

In [ ]:

def classify_node(state):

    parser = PydanticOutputParser(pydantic_object=Classify)

    format_instructions = parser.get_format_instructions()

    max_retries = 3
    attempt = 0

    agent = create_react_agent(
        model=llm,
        tools=[ddg_search_tool],
        prompt=(
            "You are an expert FDA decision-tree assistant.\n\n"
            "You MUST call the DuckDuckGoSearch tool at least once before replying.\n\n"
            "Your output MUST conform exactly to this JSON schema (no extra fields):\n"
            f"{format_instructions}\n\n"
            "Add a key `classification` with value either \"generic\" or \"fda\":\n"
            "- If the query is a greeting or unrelated to FDA topics, set `classification` to \"generic\".\n"
            "- Otherwise (drug, medical, device, risk, regulatory, decision tree) set `classification` to \"fda\".\n\n"
            "IMPORTANT: The output must be *only* the JSON object—no embedded reasoning.\n"
        )
    )

    user_msg = {"role": "user", "content": f"Classify this topic : {state['topic']}"}

    while attempt < max_retries:
        
        attempt += 1
        
        
        # Run agent and capture full assistant output (stream or no-stream)
        llm_response = agent.invoke({"messages": [user_msg]})
        # assistant_msg = llm_response["messages"][-1]
        # ai_content = assistant_msg.content

        ai_content = ""
        for step in agent.stream({"messages": [user_msg]}, stream_mode="values"):
            msg = step["messages"][-1]
            msg.pretty_print()
            # Capture only if it's an assistant message
            if isinstance(msg, AIMessage):
                ai_content = msg.content
        
        
        try :
            # Parse the final JSON into Pydantic model
            article: ArticleDraft = parser.parse(ai_content)
            return article.model_dump()
        except ValidationError as e:
            print(f"[Attempt {attempt}] Parsing failed:", e)
            # Optionally modify the prompt to highlight the error:
            prompt += "\n\nNote: Your previous output did not match the required JSON schema. Please fix it exactly."
            continue

    # If all attempts fail, raise or return empty/default
    raise RuntimeError(f"Failed to get valid ArticleDraft JSON after {max_retries} attempts.")


In [ ]:
res = classify_node({"topic": "Impact of Dumbledore in Harry's life ?"})
res

================================ Human Message =================================

Classify this topic : Impact of Dumbledore in Harry's life ?
================================== Ai Message ==================================

{"classification": "generic"}


{'classification': 'generic'}

In [ ]:
res = classify_node({"topic": "Change last node decision in the tree?"})
res

================================ Human Message =================================

Classify this topic : Change last node decision in the tree?
================================== Ai Message ==================================

{"classification": "fda"}


{'classification': 'fda'}

In [ ]:
res = classify_node({"topic": "Do not deviate from the actual path just change last node decison and tell risk"})


================================ Human Message =================================

Classify this topic : Do not deviate from the actual path just change last node decison and tell risk
================================== Ai Message ==================================

{"classification": "fda"}
